# 핸즈온ML14 합성곱 신경망을 사용한 컴퓨터 비전

### 합성곱층

-   입력층: 이미지 픽셀 정보
-   합성곱 층 1: 합성곱 층 뉴런의 수용장 안에 있는 픽셀과 해당 층의 뉴런이 연결된다. 하나의 뉴런이 모든 픽셀과 연결되지 않고, 일부에만 연결된다.
-   합성곱 층 2: 이전 층의 작은 사각 영역 안에 위치한 뉴런과 해당 층의 뉴런이 연결된다. 

### 데이터 불러오기

```
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_valid = (X_valid - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]
```

### CNN 구조

#### Conv2D

-   kernel\_size = 3 : 3X3 사이즈라는 뜻.
-   strides = (1,1)이 디폴트임 : 가로로 1칸씩 세로로 1칸씩 이라는 뜻
-   padding = 'valid'가 디폴트. valid는 패딩을 하지 않는 것, same은 왼쪽 오른쪽 위 아래 패딩을 고르게 분포시킨다는 뜻이다. 

#### Pooling 

-   MaxPool2D: 최대값으로 풀링 하는 것
    -   pool\_size = (2,2)로 디폴트. 그냥 정수만 넣으면 정사각형이라고 간주함
    -   strides =None이 디폴트
    -   padding = 'valid'가 디폴트
-   AveragePooling2D: 평균 값으로 풀링 하는 것

```
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', padding="SAME")

model = keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation='softmax'),
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
score = model.evaluate(X_test, y_test)
X_new = X_test[:10] # 새로운 이미지처럼 사용합니다
y_pred = model.predict(X_new)
```